### Import Required Package

In [354]:
import pandas as pd
import talib
import numpy as np
import json

### Import Tidal

In [355]:
import tidal as td

### Initialize Plumber

In [356]:
from pathlib import Path
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

PLUMBER_HOST = "https://dev-api.ddt-dst.cc/api/plumber/"
with open(f'{str(Path.home())}/.config/gcloud/application_default_credentials.json') as plumber_token:
    token = json.load(plumber_token)

### Load market data and compute technical indicators

In [357]:
import os
import pandas as pd

os.chdir('/home/jovyan/individualized-indicator')
# 定義你想要讀取檔案的目錄
directory = './emb'

start_date = "2022-06-01"
end_date = "2024-09-30"

# 初始化一個空的字典來存儲所有讀取的 DataFrame
dataframes = {}

# 遍歷目錄中的所有檔案
for filename in os.listdir(directory):
    if filename.endswith('.pkl'):
        # 組合完整的檔案路徑
        file_path = os.path.join(directory, filename)
        # 讀取 .pkl 檔案
        df = pd.read_pickle(file_path)
        
        # 使用檔案名稱（不包括副檔名）作為字典的鍵
        variable_name = filename.split('_')[2]
        
        # 將 DataFrame 儲存到字典中
        dataframes[variable_name] = df

keys_list = list(dataframes.keys())
# 將字符串轉換為 datetime 格式，然後進行排序
sorted_dates = sorted(pd.to_datetime(keys_list))
keys_list = [date.strftime('%Y-%m-%d') for date in sorted_dates]

total_df = pd.DataFrame()

for j in range(0,len(keys_list)):
    df_name = keys_list[j]
    selected_df = dataframes[df_name]    
    selected_df = selected_df.reset_index()
    stock_list = pd.DataFrame(selected_df.iloc[:,0].unique())
    total_df = pd.concat([total_df, stock_list], ignore_index=True)
    
unique_values_list = list(total_df.iloc[:,0].unique())
sorted_list = sorted(unique_values_list)

total_data = pd.read_parquet(
    f"{PLUMBER_HOST}stocks/tw/ohlcv",
    storage_options={
        "gcp-token": json.dumps(token),
        "start-date": start_date,
        "end-date": end_date,
        "tickers": ",".join([stock for stock in sorted_list]),
    },
)

total_data.index = total_data.index.set_levels(
    pd.to_datetime(total_data.index.levels[1]),level=1)

total_data.rename_axis(index={
    'ticker': 'instrument'
}, inplace=True)

total_data['new'] = -1
total_data.reset_index(inplace=True)

def filter_by_year_month_12(filter_data, select_year, select_mon):
    # 計算當前篩選年份和下一個年份
    next_year = select_year + 1
    
    # 構建篩選條件：當前年12月、次年1月、次年2月
    filter_data = filter_data[
        ((filter_data['datetime'].dt.year == next_year) & (filter_data['datetime'].dt.month == 1)) |
        ((filter_data['datetime'].dt.year == next_year) & (filter_data['datetime'].dt.month == 2)) |
        ((filter_data['datetime'].dt.year == next_year) & (filter_data['datetime'].dt.month == 3))
    ]
    
    return filter_data

concat_df = pd.DataFrame()

# for i in range(0,len(keys_list)-1):
for i in range(0,len(keys_list)):
    df_name = keys_list[i]
    selected_df = dataframes[df_name]
    selected_df.reset_index(inplace=True)
    stock_df = selected_df.iloc[:,0]
    stock_df_unique = list(stock_df.unique())
    result = list(set(sorted_list) - set(stock_df_unique))
    filtered_df = total_data[total_data['instrument'].isin(result)]
    filtered_df = filtered_df.rename(columns={'volume': 'vol'})
    selected_df = pd.concat([selected_df, filtered_df], ignore_index=True)
    
    select_year = int(df_name.split('-')[0])
    select_mon = int(df_name.split('-')[1])
    select_mons = [(select_mon+1)%12, (select_mon+2)%12, (select_mon+3)%12]
    select_mons = [12 if mon == 0 else mon for mon in select_mons]
    
    selected_df['datetime'] = pd.to_datetime(selected_df['datetime'])
    # 過濾出所有6月的交易資料
    filter_data = selected_df[selected_df['datetime'].dt.month.isin(select_mons)]

    # 使用時，根據 select_year 和 select_mon 動態篩選數據
        
    if select_mon == 12:
        filter_data = filter_by_year_month_12(filter_data, select_year, select_mon) 

    else:
        filter_data = filter_data[filter_data['datetime'].dt.year == select_year]
    
    concat_df = pd.concat([concat_df, filter_data], ignore_index=True)
    
# i = len(keys_list)-1
# df_name = keys_list[i]
# selected_df = dataframes[df_name]
# selected_df.reset_index(inplace=True)
# stock_df = selected_df.iloc[:,0]
# stock_df_unique = list(stock_df.unique())
# result = list(set(sorted_list) - set(stock_df_unique))
# filtered_df = total_data[total_data['instrument'].isin(result)]
# filtered_df = filtered_df.rename(columns={'volume': 'vol'})
# selected_df = pd.concat([selected_df, filtered_df], ignore_index=True)
# concat_df = pd.concat([concat_df, selected_df], ignore_index=True)

concat_df.sort_values(by=['instrument', 'datetime'], inplace=True)
concat_df.reset_index(drop=True, inplace=True)
row = np.where(concat_df.iloc[:,0] == '6251')[0]
concat_df = concat_df.drop(index=row)
concat_df = concat_df.reset_index(drop=True)
row = np.where(concat_df.iloc[:,0] == '8406')[0]
concat_df = concat_df.drop(index=row)
concat_df = concat_df.reset_index(drop=True)
row = np.where(concat_df.iloc[:,0] == '6548')[0]
concat_df = concat_df.drop(index=row)
concat_df = concat_df.reset_index(drop=True)
quote_data = concat_df.set_index(['instrument', 'datetime'])
quote_data.columns.values[4] = 'volume'

# quote_data.to_csv('./quote_data.csv')

/tmp/ipykernel_2424/4042907721.py:97: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  selected_df['datetime'] = pd.to_datetime(selected_df['datetime'])
/tmp/ipykernel_2424/4042907721.py:97: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  selected_df['datetime'] = pd.to_datetime(selected_df['datetime'])
/tmp/ipykernel_2424/4042907721.py:97: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  selected_df['

In [358]:
# i = len(keys_list)-1
# df_name = keys_list[i]
# selected_df = dataframes[df_name]
# selected_df.reset_index(inplace=True)
# stock_df = selected_df.iloc[:,0]
# stock_df_unique = list(stock_df.unique())
# result = list(set(sorted_list) - set(stock_df_unique))
# filtered_df = total_data[total_data['instrument'].isin(result)]
# filtered_df = filtered_df.rename(columns={'volume': 'vol'})
# selected_df = pd.concat([selected_df, filtered_df], ignore_index=True)
# concat_df = pd.concat([concat_df, selected_df], ignore_index=True)

### Strategy Lake API

In [359]:
data_api_url = ("https://dev-api.ddt-dst.cc/api/plumber/nextron/industry_prod_df")
df = pd.read_parquet(
    data_api_url,
    storage_options={
        "gcp-token": json.dumps(token),
    }
)

### Load benchmark data

In [360]:
benchmark_inst = "Y9999"
benchmark_data = pd.read_parquet(
    f"{PLUMBER_HOST}stocks/tw/ohlcv",
    storage_options={
        "gcp-token": json.dumps(token),
        "start-date": start_date,
        "end-date": end_date,
        "tickers": benchmark_inst,
    },
)
benchmark_data.index.set_levels(
    pd.to_datetime(benchmark_data.index.levels[1]),
    level=1,
    inplace=True,
)
benchmark_data.rename_axis(index={
    'ticker': 'instrument'
}, inplace=True)

benchmark_data

/tmp/ipykernel_2424/1318227356.py:11: FutureWarning: inplace is deprecated and will be removed in a future version.
  benchmark_data.index.set_levels(


open      high       low     close      volume
instrument datetime                                                      
Y9999      2022-06-01  16718.91  16811.04  16649.91  16675.09  5693916000
           2022-06-02  16605.46  16617.26  16540.55  16552.57  4836099000
           2022-06-06  16570.89  16654.12  16509.01  16605.96  5418061000
           2022-06-07  16593.75  16593.75  16465.87  16512.88  4732466000
           2022-06-08  16538.67  16702.99  16538.67  16670.51  4815479000
...                         ...       ...       ...       ...         ...
           2024-09-24  22273.79  22431.78  22154.55  22431.78  7862076000
           2024-09-25  22569.29  22785.32  22569.29  22761.60  9143783000
           2024-09-26  22841.42  22961.77  22769.24  22858.81  9160464000
           2024-09-27  22978.97  23079.21  22806.83  22822.79  9742471000
           2024-09-30  22643.91  22691.23  22224.54  22224.54  9221061000

[569 rows x 5 columns]

### Initialize Tidal
1. Initialize Tidal object
2. Add Quote data (pd.DataFrame)
3. Set strategy object (td.BaseStrategy)
4. Add metric objects (td.BaseMetic)

In [361]:
# Initialize Tidal object
tidal = td.Tidal(init_cash=50000000, slip_ticks=1, stock_config=td.StockConfig.TW, load_configs=True, reqMem="1000Mi", ignore_volume_size=True)

# Add quote data
tidal.add_quote(quote_data)

# Set strategy object
tidal.set_strategy(td.strategy.TopkDropout(10, 1, 'new'))

# Set metric objects
tidal.add_metric(td.metric.AccountInfo())
tidal.add_metric(td.metric.AdditionalInfo())
tidal.add_metric(td.metric.PositionInfo())
tidal.add_metric(td.metric.Portfolio(benchmark_data.loc[benchmark_inst]))

### Stock Config

In [362]:
tidal.exchange.stock_config

InstConfig {Margin:0.0, Tick Size:0.002, Tick Value:0.002, Trade Unit:1000, Commission:0.0004275, Min Commission:20.0, Transaction Tax:0.003}

### Instrument Configs

In [363]:
tidal.exchange.inst_configs

{'MTX': InstConfig {Margin:46000.0, Tick Size:1.0, Tick Value:50.0, Trade Unit:1, Commission:20.0, Min Commission:0.0, Transaction Tax:2e-05},
 'TX': InstConfig {Margin:184000.0, Tick Size:1.0, Tick Value:200.0, Trade Unit:1, Commission:40.0, Min Commission:0.0, Transaction Tax:2e-05},
 'NQ': InstConfig {Margin:16500.0, Tick Size:0.25, Tick Value:5.0, Trade Unit:1, Commission:1.85, Min Commission:0.0, Transaction Tax:0.0}}

### Config Modification

In [364]:
# Set commission to 77% off
tidal.exchange.set_stock_config(commission=0.001425 * 0.23)
tidal.exchange.stock_config

InstConfig {Margin:0.0, Tick Size:0.002, Tick Value:0.002, Trade Unit:1000, Commission:0.00032775, Min Commission:20.0, Transaction Tax:0.003}

### Add New Instrument Config

In [365]:
# Add MGC config
tidal.exchange.set_config(instrument='MGC', margin=787., tick_size=0.1, tick_value=1., trade_unit=1, commission=2., min_commission=0., transaction_tax=0.)
tidal.exchange.inst_configs

{'MTX': InstConfig {Margin:46000.0, Tick Size:1.0, Tick Value:50.0, Trade Unit:1, Commission:20.0, Min Commission:0.0, Transaction Tax:2e-05},
 'TX': InstConfig {Margin:184000.0, Tick Size:1.0, Tick Value:200.0, Trade Unit:1, Commission:40.0, Min Commission:0.0, Transaction Tax:2e-05},
 'NQ': InstConfig {Margin:16500.0, Tick Size:0.25, Tick Value:5.0, Trade Unit:1, Commission:1.85, Min Commission:0.0, Transaction Tax:0.0},
 'MGC': InstConfig {Margin:787.0, Tick Size:0.1, Tick Value:1.0, Trade Unit:1, Commission:2.0, Min Commission:0.0, Transaction Tax:0.0}}

### Start Backtesting

In [366]:
tidal.backtest()
logs = tidal.get_logs()
for log in logs:
    print(log)

Tidal Backtesting: 100%|██████████| 548/548 [01:32<00:00,  5.91it/s, cash=4.51e+5, pnl=-1.39e+6, position_cost=7.29e+7, value=7.19e+7]

2023-06-30 - 訂單未能執行: 6669, 可能是由於現金不足或其他原因


### Metric - AccountInfo

In [367]:
account_info = tidal.metrics["AccountInfo"].report
print(account_info)

                    cash    trade_cost  position_cost     slip_cost  \
datetime                                                              
2022-07-01  5.000000e+07  0.000000e+00   0.000000e+00  0.000000e+00   
2022-07-04  5.000000e+07  0.000000e+00   0.000000e+00  0.000000e+00   
2022-07-05  2.218451e+06  1.565527e+04   4.776589e+07  7.484408e+05   
2022-07-06  3.465033e+05  3.414575e+04   4.962966e+07  7.715640e+05   
2022-07-07  4.496663e+05  5.822922e+04   4.935657e+07  7.771039e+05   
...                  ...           ...            ...           ...   
2024-09-24  6.083647e+05  9.033702e+06   7.320480e+07  9.518958e+06   
2024-09-25  4.213243e+05  9.060706e+06   7.265180e+07  9.610294e+06   
2024-09-26  7.081761e+05  9.108502e+06   7.229057e+07  9.550147e+06   
2024-09-27  4.510640e+05  9.139032e+06   7.287173e+07  9.523429e+06   
2024-09-30  4.510640e+05  9.139032e+06   7.287173e+07  9.523429e+06   

                     pnl         value  max_drawdown      turnover  
datetim

### Metric - PositionInfo

In [368]:
position_df = tidal.metrics["PositionInfo"].report
print(position_df)

                       quantity      price   commission  slip_cost  \
instrument datetime                                                  
1102       2022-07-26  161000.0  37.342700  1970.490258 -20157.200   
           2022-07-27  161000.0  37.342700  1970.490258 -20157.200   
           2022-07-29  155000.0  37.121194  1885.803067  81498.101   
           2022-08-01  155000.0  37.121194  1885.803067  81498.101   
           2022-08-02  155000.0  37.121194  1885.803067  81498.101   
...                         ...        ...          ...        ...   
9945       2024-09-24  131000.0  48.496800  2082.222232  38880.800   
           2024-09-25  131000.0  48.496800  2082.222232  38880.800   
           2024-09-26  131000.0  48.496800  2082.222232  38880.800   
           2024-09-27  131000.0  48.496800  2082.222232  38880.800   
           2024-09-30  131000.0  48.496800  2082.222232  38880.800   

                              pnl  now_price  
instrument datetime                       

In [369]:
pi_report = tidal.metrics['PositionInfo'].report
pi_report.iloc[pi_report.index.get_level_values('datetime') == '2022-06-14']

,,quantity,price,commission,slip_cost,pnl,now_price
instrument,datetime,,,,,,


### Metric - AdditionalInfo

In [370]:
tidal.metrics['AdditionalInfo'].report

new
instrument datetime            
1101       2022-07-01  0.000423
           2022-07-04 -0.002125
           2022-07-05  0.000366
           2022-07-06 -0.003043
           2022-07-07  0.001127
...                         ...
9958       2024-09-24  0.000538
           2024-09-25  0.000034
           2024-09-26  0.000216
           2024-09-27  0.000183
           2024-09-30  0.000111

[342085 rows x 1 columns]

### Strategy Lake Submit

In [371]:
# group = dict(zip(df["coid"].astype(str), df["tejind4_c"]))
# submit_lake_backtest_result = tidal.submit_lake_backtest(account_info=account_info, position_df=position_df, benchmark_info=benchmark_data, group=group,lake_env="dev",strategy_id=19)
# print(submit_lake_backtest_result)

update lake backtest data success


### Trade Report

In [372]:
tidal.trade_report

,win_num,lose_num,trade_num,pos_num,win_rate,profit,loss,trade_cost,pnl
instrument,,,,,,,,,
6139,2.0,2.0,4.0,0.0,0.500000,7.770431e+06,-4.075746e+05,1.179057e+05,7.362856e+06
2392,2.0,0.0,2.0,0.0,1.000000,4.681635e+06,0.000000e+00,6.008530e+04,4.681635e+06
3338,1.0,0.0,1.0,0.0,1.000000,4.387503e+06,0.000000e+00,6.440675e+04,4.387503e+06
3712,2.0,1.0,3.0,0.0,0.666667,2.684918e+06,-3.830769e+05,8.747638e+04,2.301842e+06
1522,2.0,0.0,2.0,0.0,1.000000,2.129302e+06,0.000000e+00,5.381837e+04,2.129302e+06
...,...,...,...,...,...,...,...,...,...
6741,2.0,4.0,6.0,0.0,0.333333,3.691529e+05,-1.858729e+06,1.878479e+05,-1.489576e+06
6288,0.0,3.0,3.0,0.0,0.000000,0.000000e+00,-1.503241e+06,7.077994e+04,-1.503241e+06
6290,1.0,3.0,4.0,0.0,0.250000,2.214049e+05,-2.224065e+06,1.037873e+05,-2.002660e+06


### Traded instruments

In [373]:
tidal.account.trades.keys()

dict_keys(['2834', '2542', '3552', '6274', '8436', '6712', '2207', '6683', '2812', '4904', '8081', '5371', '8924', '5340', '1102', '5478', '5609', '6472', '6409', '1537', '2640', '6533', '6591', '3551', '5522', '5274', '3008', '3669', '4971', '1580', '2409', '5434', '6669', '6568', '3617', '2723', '3592', '3029', '4104', '6470', '6605', '4966', '1590', '3455', '2420', '6125', '3529', '6188', '6235', '6515', '2458', '2233', '5607', '8086', '8210', '3260', '2474', '3661', '3006', '1519', '6612', '6290', '6284', '3675', '3443', '6510', '1712', '6176', '2059', '6288', '3038', '8422', '6139', '4979', '4557', '8938', '3416', '3227', '3044', '6205', '1717', '8048', '4137', '2453', '2916', '6577', '5508', '6691', '4138', '6776', '6538', '5511', '3356', '6170', '1522', '8112', '3213', '6523', '6585', '5236', '6189', '6788', '2392', '3338', '9927', '3712', '3622', '4933', '2548', '6829', '8109', '1342', '6741', '3624', '6579', '2352', '4919', '5309', '9945', '8255', '3022', '6672', '3289'])

### Trading History

In [374]:
# tidal.account.trades['3015']

### Plot chart by using Plotly

In [375]:
# tidal.analyzer.inst_chart(instrument='3015', metric_name='AdditionalInfo', plot_type=td.PlotType.LINE, scale=1.0)

### Tidal Dashboard

In [376]:
tidal.tdboard()

 * Serving Flask app 'tidal.tdboard'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:33031
 * Running on http://10.136.13.64:33031
INFO:werkzeug:Press CTRL+C to quit


In [ ]:
tidal.account.position_history